In [4]:
import geopandas as gpd
import sumolib

In [13]:
# read the buildings 
bld = gpd.read_file('C:/Users/barguzin/Documents/Github/rwmp_pipelines/resid_centr_mc.geojson')
print(bld.crs)
print(bld.shape)

# parse the net
net = sumolib.net.readNet('C:/Users/barguzin/Documents/Github/rwmp_pipelines/study_area.net.xml')

epsg:4326
(1282, 6)


In [32]:
all_dist = []
all_edges = []
all_ids = []

lng, lat = bld.geometry.x, bld.geometry.y 

radius=50 # meters

xlon, xlat = net.convertLonLat2XY(lng, lat)

for x,y in zip(xlon, xlat): 
    edges = net.getNeighboringEdges(x, y, radius)

    if len(edges) > 0: 
        distancesAndEdges = sorted([(dist, edge) for edge, dist in edges], key=lambda x: x[0])
        dist, closestEdge = distancesAndEdges[0]

        all_dist.append(dist)
        all_edges.append(closestEdge)
        all_ids.append(closestEdge.getID())

In [24]:
print(bld.shape)
print(len(all_dist))
print(len(all_edges))

(1282, 6)
1242
1242


In [25]:
print(all_edges[:5])

[<edge id="-16262970#1" from="165437782" to="165476353"/>, <edge id="-678722520" from="6355141679" to="6355141889"/>, <edge id="-679909983#0" from="6366185189" to="6366185191"/>, <edge id="-679909981" from="6366185189" to="6366184872"/>, <edge id="16253433#2" from="566126288" to="6355141622"/>]


In [30]:
print(closestEdge)
closestEdge.getID()

<edge id="924683558" from="8581835023" to="8581835028"/>


'924683558'

In [38]:
import pandas as pd

df = pd.DataFrame(list(zip(all_ids, all_dist, all_edges)), columns=['id', 'dist', 'from_to'])
df

,id,dist,from_to
0,-16262970#1,16.347742,"<edge id=""-16262970#1"" from=""165437782"" to=""16..."
1,-678722520,11.383053,"<edge id=""-678722520"" from=""6355141679"" to=""63..."
2,-679909983#0,10.340639,"<edge id=""-679909983#0"" from=""6366185189"" to=""..."
3,-679909981,7.984477,"<edge id=""-679909981"" from=""6366185189"" to=""63..."
4,16253433#2,22.724856,"<edge id=""16253433#2"" from=""566126288"" to=""635..."
...,...,...,...
1237,924683558,8.126412,"<edge id=""924683558"" from=""8581835023"" to=""858..."
1238,-886568238#1,37.294320,"<edge id=""-886568238#1"" from=""8563538342"" to=""..."
1239,-627887422#3,36.524892,"<edge id=""-627887422#3"" from=""165621894"" to=""1..."
1240,-922463717,20.488067,"<edge id=""-922463717"" from=""8563538343"" to=""85..."


In [ ]:
# to generate traffic from OD in SUMO 
# Prep inputs: TAZ and OD cost matrix

# TAZ
# 1. import shapes (primary highways buffer) via 'polyconvert' tool (These are TAZs)
# 2. assign edges to TAZ () via 'edgesInDistrict.py'

# OD 
# 1. create routes 